DATA CLEANING/LOADING:

In [2]:
!pip install -q pandas matplotlib seaborn numpy KDTree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.spatial import KDTree

In [3]:
tech_salaries = pd.read_csv('/Users/annamezhlauk/Desktop/ECO225Project/Data/Tech_salaries.csv')
startups_main = pd.read_csv('/Users/annamezhlauk/Desktop/ECO225Project/Data/Startups/objects.csv', low_memory = False)
startups_offices = pd.read_csv('/Users/annamezhlauk/Desktop/ECO225Project/Data/Startups/offices.csv', low_memory = False)

In [4]:
tech_salaries = tech_salaries.drop([44, 53, 267, 281, 522, 586, 646, 789, 1303, 1422, 1423, 1424, 1555, 1595, 1640, 1642], axis=0)
# dropping observations with "spam" position names


tech_salaries = tech_salaries.drop([1448, 1132, 860, 766, 689, 677, 387, 21], axis=0)
# dropping observations where compensation isn't in USD

tech_salaries = tech_salaries[(tech_salaries['annual_base_pay'] >= 10000) & (tech_salaries['annual_base_pay'] <= 5000000)].reset_index(drop=True)
# dropping improbable salaries that could be data entry mistakes

tech_salaries.drop(['location_latitude', 'location_longitude', 'location_name', 'location_state', 'location_country'], axis=1, inplace=True)

In [5]:
startups_main.rename(columns = {"normalized_name": "employer_name"}, inplace = True)
startups_main.rename(columns = {"id": "object_id"}, inplace = True)

startups_main = startups_main[startups_main['funding_total_usd'] >= 1000].reset_index(drop=True)
# dropping improbable total funding amounts, as well as missing values which are sometimes entered as 0

In [6]:
startups_offices = startups_offices[~((startups_offices['latitude'] == 0) & (startups_offices['longitude'] == 0))].reset_index(drop=True)

In [7]:
startups_salaries = tech_salaries.merge(startups_main, how="left", on= "employer_name")
startups_salaries_geo = startups_salaries.merge(startups_offices, how="left", on="object_id")

In [8]:
startups_salaries_geo_clean = startups_salaries_geo.dropna(subset=['category_code', 'latitude', 'longitude'])

In [29]:

# Step 2: Convert latitude and longitude to radians (using .loc to avoid SettingWithCopyWarning)
startups_salaries_geo_clean = startups_salaries_geo_clean.copy()  # Ensure it's a copy if previously sliced
startups_salaries_geo_clean.loc[:, 'lat_radians'] = np.radians(startups_salaries_geo_clean['latitude'])
startups_salaries_geo_clean.loc[:, 'lon_radians'] = np.radians(startups_salaries_geo_clean['longitude'])

# Step 3: Prepare industry coordinates grouped by category_code
industry_coords = startups_salaries_geo_clean.groupby('category_code')[['lat_radians', 'lon_radians']]

# Step 4: Define the Haversine function to calculate the distance between two points
def haversine(lat1, lon1, lat2, lon2):
    # Haversine formula to calculate distance between two points
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    radius_earth_miles = 3960  # Radius of the Earth in miles
    return radius_earth_miles * c

# Step 5: Define the function to count competitors within a radius
def count_competitors(index, lat, lon, industry, radius_miles):
    # Get the industry's coordinates
    industry_df = industry_coords.get_group(industry)  # Use .get_group to avoid KeyError
    industry_df['index'] = industry_df.index  # Ensure the 'index' column exists
    
    # Calculate distances and count competitors within the radius
    distances = industry_df.apply(
        lambda row: haversine(lat, lon, row['lat_radians'], row['lon_radians']),
        axis=1
    )
    
    
    # Get the indices of companies within the radius (excluding itself)
    indices = industry_df[distances < radius_miles].index
    
    
    actual_index = index
    
    return len(indices) - 1 if actual_index in indices else len(indices)  # Exclude itself if present

# Step 6: Apply the function to count competitors within the radius for each row
radius_miles = 6000
startups_salaries_geo_clean.loc[:, 'competitors_within_radius'] = startups_salaries_geo_clean.apply(
    lambda row: count_competitors(row.name, row['latitude'], row['longitude'], row['category_code'], radius_miles), 
    axis=1
)

# Step 7: Print the final DataFrame (for checking results)
print(startups_salaries_geo_clean[['employer_name', 'category_code', 'competitors_within_radius']])


     employer_name     category_code  competitors_within_radius
0           opower         cleantech                          7
1           opower         cleantech                          7
8          knewton         education                          3
9          amplify         education                          3
22         liveops  public_relations                          1
...            ...               ...                        ...
4113      facebook            social                         72
4121       comcast       games_video                          9
4122      facebook            social                         47
4123      facebook            social                         48
4124      facebook            social                         72

[2875 rows x 3 columns]


In [33]:
print(startups_salaries_geo_clean['category_code'].unique())

['cleantech' 'education' 'public_relations' 'software' 'enterprise'
 'games_video' 'social' 'web' 'ecommerce' 'search' 'mobile' 'music'
 'network_hosting' 'advertising' 'transportation' 'nonprofit' 'hardware'
 'other' 'finance' 'security' 'health' 'semiconductor' 'biotech' 'news'
 'analytics' 'real_estate']


In [31]:
# Test with a small subset of the dataset
small_test_df = startups_salaries_geo_clean.head(10)
small_test_df.loc[:, 'competitors_within_radius'] = small_test_df.apply(
    lambda row: count_competitors(row.name, row['latitude'], row['longitude'], row['category_code'], radius_miles), 
    axis=1
)

print(small_test_df[['employer_name', 'category_code', 'competitors_within_radius']])


    employer_name     category_code  competitors_within_radius
0          opower         cleantech                          7
1          opower         cleantech                          7
8         knewton         education                          3
9         amplify         education                          3
22        liveops  public_relations                          1
33  social tables          software                         35
38         twilio        enterprise                         15
42        comcast       games_video                          9
43       linkedin            social                         47
44    nara logics               web                          0


In [10]:
startups_salaries_geo_clean.describe()

,index,salary_id,total_experience_years,employer_experience_years,annual_base_pay,signing_bonus,annual_bonus,entity_id,logo_width,logo_height,...,relationships,id,office_id,latitude,longitude,created_at_y,updated_at_y,lat_radians,lon_radians,competitors_within_radius
count,2875.000000,2875.000000,2785.000000,2790.000000,2.875000e+03,2464.000000,2768.000000,2875.00000,2875.000000,2875.000000,...,2875.000000,2875.000000,2875.000000,2875.000000,2875.000000,0.0,0.0,2875.000000,2875.000000,2875.000000
mean,840.864348,1712.659826,6.211131,2.619659,1.497907e+05,26814.304789,26484.141618,1345.50713,792.801739,277.948174,...,944.232000,4921.805565,5916.828174,38.663372,-49.345309,NaN,NaN,0.674803,-0.861238,57.669565
std,421.906265,818.991878,4.717589,2.356956,1.461054e+05,73501.627873,32669.073543,11004.06307,269.492097,106.541714,...,286.356364,7484.234067,8655.973599,20.976676,68.965634,NaN,NaN,0.366112,1.203677,143.948763
min,0.000000,1.000000,0.000000,0.000000,1.060000e+04,0.000000,0.000000,5.00000,100.000000,28.000000,...,1.000000,4.000000,5.000000,-37.879234,-122.500111,NaN,NaN,-0.661117,-2.138030,0.000000
25%,430.000000,936.000000,2.500000,1.000000,1.140000e+05,0.000000,16375.000000,59.00000,800.000000,274.000000,...,1046.000000,4233.000000,5140.000000,34.019282,-104.898419,NaN,NaN,0.593748,-1.830823,0.000000
50%,938.000000,1915.000000,5.000000,2.000000,1.300000e+05,7500.000000,20400.000000,59.00000,800.000000,274.000000,...,1046.000000,4246.000000,5153.000000,41.889474,-77.027101,NaN,NaN,0.731109,-1.344377,0.000000
75%,1196.000000,2381.000000,8.000000,4.000000,1.520000e+05,25000.000000,30000.000000,59.00000,800.000000,274.000000,...,1046.000000,4273.000000,5180.000000,50.063281,9.195978,NaN,NaN,0.873769,0.160500,0.000000
max,1648.000000,3285.000000,25.000000,11.000000,5.000000e+06,500000.000000,750000.000000,179530.00000,2100.000000,1931.000000,...,1046.000000,99758.000000,114165.000000,65.617491,145.073608,NaN,NaN,1.145241,2.532012,681.000000
